In [1]:
#!/ usr/bin/env python3 .7
# Copyright 2020 , Gurobi Optimization , LLC
# This example considers the following separable , convex problem :
#
# minimize   f(x) - y + g(z)
# subject to x + 2 y + 3 z <= 4
#            x + y >= 1
#            x, y, z <= 1
#
# where f(u) = exp(-u) and g(u) = 2 u^2 - 4 u, for all real u. It
# formulates and solves a simpler LP model by approximating f and
# g with piecewise-linear functions . Then it transforms the model
# into a MIP by negating the approximation for f, which corresponds
# to a non-convex piecewise-linear function, and solves it again .

In [3]:
import gurobipy as gp
from math import exp

In [5]:
def f(u):
    return exp(-u)

def g(u):
    return 2 * u * u - 4 * u

In [6]:
m = gp.Model()

Using license file /Users/yj/gurobi.lic


In [7]:
lb = 0.0
ub = 1.0

In [8]:
x = m.addVar(lb, ub, name='x')
y = m.addVar(lb, ub, name='y')
z = m.addVar(lb, ub, name='z')

Core part: Approximation of objective function

In [9]:
m.setObjective(-y)

In [10]:
npts = 101
ptf = []
ptf = []
ptg = []

for i in range(npts):
    ptu.append(lb + (ub - lb) * i / (npts - 1))
    ptf.append(f(ptu[i]))
    ptg.append(g(ptu[i]))

In [12]:
# Approximation
m.setPWLObj(x, ptu, ptf)
m.setPWLObj(z, ptu, ptf)

In [13]:
m.addConstr(x + 2 * y + 3 * z <= 4, 'c0')
m.addConstr(x + y >= 1, 'c1')

<gurobi.Constr *Awaiting Model Update*>

In [14]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x91ac9429
Model has 2 piecewise-linear objective terms
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve time: 0.07s
Presolved: 2 rows, 3 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.6424112e-01   5.000000e-01   0.000000e+00      0s
       1    8.4418704e-02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.13 seconds
Optimal objective  8.441870438e-02


In [15]:
print('IsMIP: %d' % m.IsMIP)
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))
print('Obj: %g' % m.ObjVal)
# print('')

IsMIP: 0
x 1
y 1
z 0.333333
Obj: 0.0844187


In [16]:
# Negate piecewise - linear objective function for x
for i in range(npts):
    ptf[i] = - ptf[i]

In [17]:
m.setPWLObj(x, ptu, ptf)

In [18]:
# Optimize model as a MIP
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model has 2 piecewise-linear objective terms
Variable types: 3 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -0.3678794
Presolve time: 0.02s
Presolved: 202 rows, 302 columns, 603 nonzeros
Variable types: 203 continuous, 99 integer (99 binary)

Root relaxation: objective -1.486577e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      -1.4865772   -1.48658  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.13 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: -1.48658 -0.367879 
No other solutions bette

In [19]:
print('IsMIP: %d' % m.IsMIP)
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))
print('Obj: %g' % m.ObjVal)

IsMIP: 1
x 0
y 1
z 0.666667
Obj: -1.48658
